<a href="https://colab.research.google.com/github/Akr-ds/GPT-FROM-SCRATCH/blob/main/gpt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import mmap
import random
import pickle
import argparse

# Hyperparameters
block_size = 128
batch_size = 32
learning_rate = 0.0001
max_iters = 5000
eval_iters = 100
n_embd = 256
n_head = 8
n_layers = 6
dropout = 0.1

# Check device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Argument parser (for demonstration)
parser = argparse.ArgumentParser(description='This is a demonstration program')

# Reading and encoding text
chars = ""
with open("/content/wizard_of_oz.txt", 'r', encoding='utf-8') as f:
    text = f.read()
    chars = sorted(list(set(text)))

vocab_size = len(chars)
string_to_int = {ch: i for i, ch in enumerate(chars)}
int_to_string = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

# Functions to get random chunks and batches
def get_random_chunk(split):
    filename = "/content/train_split.txt" if split == 'train' else "/content/val_split.txt"
    with open(filename, 'rb') as f:
        with mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ) as mm:
            file_size = len(mm)
            if file_size <= block_size * batch_size:
                raise ValueError("File size is smaller than the required chunk size.")
            start_pos = random.randint(0, file_size - block_size * batch_size)
            mm.seek(start_pos)
            block = mm.read(block_size * batch_size - 1)
            decoded_block = block.decode('latin-1', errors='ignore').replace('\r', '')
            data = torch.tensor(encode(decoded_block), dtype=torch.long)
    return data

def get_batch(split):
    data = get_random_chunk(split)
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i + block_size] for i in ix])
    y = torch.stack([data[i + 1:i + block_size + 1] for i in ix])
    return x.to(device), y.to(device)

# Define the Head class for multi-head attention
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        v = self.value(x)
        weights = q @ k.transpose(-2, -1) * k.shape[-1] ** -0.5
        weights = weights.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        weights = F.softmax(weights, dim=-1)
        weights = self.dropout(weights)
        output = weights @ v
        return output

# Define the MultiHeadAttention class
class MultiHeadAttention(nn.Module):
    def __init__(self, n_head, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(n_head)])
        self.proj = nn.Linear(n_head * head_size, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

# Define the FeedForward class
class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)

# Define the Block class for transformer blocks
class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x + y)
        y = self.ffwd(x)
        x = self.ln2(x + y)
        return x

# Define the main GPT-like model class
class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layers)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, index, targets=None):
        B, T = index.shape
        tok_emb = self.token_embedding_table(index)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            index_cond = index[:, -block_size:]
            # get the predictions
            logits, loss = self.forward(index_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

# Initialize model and move to device
model = GPTLanguageModel(vocab_size).to(device)

# Function to estimate loss
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# Optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# Training loop
for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(f"Final loss: {loss.item():.3f}")


step: 0, train loss: 4.434, val loss: 4.429
step: 100, train loss: 2.644, val loss: 2.665
step: 200, train loss: 2.527, val loss: 2.569
step: 300, train loss: 2.454, val loss: 2.515
step: 400, train loss: 2.411, val loss: 2.479
step: 500, train loss: 2.380, val loss: 2.450
step: 600, train loss: 2.298, val loss: 2.364
step: 700, train loss: 2.203, val loss: 2.288
step: 800, train loss: 2.150, val loss: 2.236
step: 900, train loss: 2.085, val loss: 2.175
step: 1000, train loss: 2.009, val loss: 2.111
step: 1100, train loss: 1.959, val loss: 2.068
step: 1200, train loss: 1.910, val loss: 2.025
step: 1300, train loss: 1.869, val loss: 1.989
step: 1400, train loss: 1.819, val loss: 1.946
step: 1500, train loss: 1.770, val loss: 1.907
step: 1600, train loss: 1.747, val loss: 1.896
step: 1700, train loss: 1.721, val loss: 1.850
step: 1800, train loss: 1.681, val loss: 1.834
step: 1900, train loss: 1.662, val loss: 1.814
step: 2000, train loss: 1.622, val loss: 1.807
step: 2100, train loss: 1

In [10]:
logits,loss

(tensor([[ 0.7120,  4.2509, -2.0725,  ...,  1.6494, -2.3472, -6.3490],
         [-5.4692, -2.2357, -6.2730,  ...,  1.3493, -3.7774, -6.7960],
         [ 2.2410,  5.0809, -4.3517,  ...,  0.3684, -1.9678, -7.2613],
         ...,
         [-5.3533, -2.9257, -5.9271,  ...,  1.1979, -4.0094, -6.2755],
         [ 0.1913, -0.2643,  0.2239,  ...,  3.8173, -2.7718, -1.8595],
         [-2.0542, -1.3142, -3.0003,  ...,  0.2616, -1.3140, -3.5611]],
        device='cuda:0', grad_fn=<ViewBackward0>),
 tensor(1.2976, device='cuda:0', grad_fn=<NllLossBackward0>))

In [11]:
prompt = " uncle henry"
context = torch.tensor(encode(prompt), dtype=torch.long,device=device)
gen = decode(model.generate(context.unsqueeze(0), max_new_tokens=100)[0].tolist())
gen

' uncle henry and what too very in about the\ngrows, soming interidly. They walking so that tever glass were and\nt'

In [ ]:
chars = ""
with open("/content/Introduction to Neuroscience.txt", 'r', encoding='latin-1') as f:
    text = f.read()
    chars = sorted(list(set(text)))

    string_into_int={ch:i for i,ch in enumerate(chars)}
    int_to_string={i:ch for i,ch in enumerate(chars)}
    encode=lambda s:[string_into_int[c] for c in s]
    decode=lambda l:''.join([ int_to_string[i] for i in l])

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import mmap
import random
import pickle
import argparse

# Hyperparameters
block_size = 64
batch_size = 128
learning_rate = 0.1
max_iters = 90
eval_iters = 18
n_embd = 384
n_head = 8
n_layers = 8
dropout = 0.05

In [ ]:
torch.randint(len(text)-block_size,(batch_size,))

tensor([   9174,  292113,  169251,  898667,  468466,  171692,  230941,  488348,
         498748,  616619,  686314,  412377,  180236,   48697,  359494,  599405,
         440534,  231943,  173731,  156084,  551028,  890440,  661269,  405831,
         837394,  985440,  242809,   46132,  880919,  940540,  508795,  943466,
         815890,   44045,  797567,  152946,  609730,  489689,  472278, 1018768,
         502223,   91808,  600298,  945125,  851339,  888026,  591576,  335650,
         131840,  360089,  211538,  297703,  479017,  703069, 1017226,  581283,
         700123,  723533,  452813, 1008073,  680458,   91135,  618379, 1032544,
         735079,  490454,  800718,  240708, 1024553,  662899,  593536,  315434,
         581076,  556582, 1008618,  866992,  911022,  750116,  164983,  997202,
          82125,  161583,  163380,  919281,  200888,  915490,  738674,  613002,
           9518,  758542,  283901,  633085,  867439,  496918,  335603,   54472,
         572422,  789556,  468186,  7588

In [ ]:
len(text)

1039654

In [ ]:
torch.tril((torch.ones(block_size,block_size)))

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [1., 1., 0.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        ...,
        [1., 1., 1.,  ..., 1., 0., 0.],
        [1., 1., 1.,  ..., 1., 1., 0.],
        [1., 1., 1.,  ..., 1., 1., 1.]])